# MHPI *DeltaModel* + *hydroDL2.0* Tutorial: **$\delta$ HBV**
Last Revision: 7 Nov. 2024

---


In this tutorial, we’ll walk through a basic implementation of the generic differentiable modeling framework *DeltaModel*
using the hydrology model [HBV](https://en.wikipedia.org/wiki/HBV_hydrology_model) from *hydroDL2.0* as a physics backbone.
The resulting model we call $\delta$ HBV (citation).
    
In general, a differentiable model is composed of a coupled physics model and neural network.
Physics model includes parameters for which True values are unknown, but approximated in practice.
By coupling a neural network, we can learn a set of parameters that can then be fed to the physics model
alongside other known input variables to make predictions.

In our case, we use physics model HBV and an LSTM parameterization network as the core of the differentiable model.
Since the HBV model uses input forcing variables precipitation, temperature, and potential evapotranspiration (PET)
with dimensions of days x sites, the LSTM will learn parameters at every site for every day. The structure of the
differentiable model, therefore, looks like so:

    Differentiable Model $\delta$ HBV: 
    (inputs weather forcings, $x$, and basin attributes $A$)

        learned parameters = LSTM($x$, $A$)
        hydrologic predictions = HBV($x$, learned_parameters)
    
Currently, $\delta$ HBV is setup to train and make hydrologic predictions on **streamflow**, but it can
also be reconfigured without much effort to predict percolation, recharge, and groundwater flow, among others.


After showing an example implementation, we'll demonstrate how to train the model and expose critical details of the process.


---


## Setup
**Code**: Before beginning, please see `examples/differentiable hydrology/setup_guide.md`
for steps on how to setup DeltaModel, hydroDL2, and the included Conda ENV (`envs/deltamod_env.yaml`)

**ENV** For a quick start, we recommend using the `deltamod` ENV, which includes a minimal list of
all core DeltaModel dependencies.

**Data** Preconfigured CAMELS data files can be shared on request, but will eventually be made available through our
data engine `hydro_data_dev` along with expanded instructions on how to access.


## 1. Creating a Model
For this example, we demonstrate how to get DeltaModel running with minimal setup using the HBV implementation
from hydroDL2. Note, there are two versions of $\delta$ HBV which can be run here:

- **$\delta$ HBV 1.0**: Uses the standard version of the HBV model (citation).
- **$\delta$ HBV 1.1p**: An enhanced version of the HBV model, incorporating modifications like a capillary rise module to
improve prediction performance over 1.0 (citation).

Switching between these two versions requires only a simple configuration file change, as outlined below.



- #### **Set Model and Experiment Configurations**
    
    Define your desired model settings and experimental parameters in a YAML configuration file. For this tutorial, two
    configuration files have been premade for running both versions of $\delta$ HBV. Each is configured to
    use 531 CAMELS basins with 34 years of data available as inputs. For temporal tests, we have further configured
    both files to run a 9-year training phase from 1999-2008 and 10-year test phase from 1989-1999 to reproduce benchmark
    results of (citation):
    -  **$\delta$ HBV 1.0**: `example/conf/dhbv_config.yaml`
    - **$\delta$ HBV 1.1p**: `example/conf/dhbv_v1_1p_config.yaml` 

- #### **Building the Model**

    Currently, differentiable models can be built with DeltaModel in two ways of varying flexibility:
    - **Implicit**: This method is best for small-scale experiments and sharing final products.
    
        Add/change modules in DeltaModel to create your own differentiable model, and add to/change
        configuration settings in `deltaModel/conf/config.yaml` to reflect the needs of your product. (Modules including
        trainers, physics models, neural networks, loss functions, are designed to be hot-swappable per user needs. The
        differentiable model modality will eventually also be made more flexible to meet diverse modeling needs. See
        `docs/getting_started_guide` for more info on making the DeltaModel your own.) 

        With these pieces in place, simply run the model in your terminal using
        ```shell
        cd ./deltaModel
        python __main__.py
        ```

    - **Explicit** (Code block below): This method is best for exploratory research and prototyping.

        This approach is similar in that we still use a configuration file to hold model and experiment settings (though
        a dictionary object can just as well be used). However, here we can expose the fundamental steps in the
        modeling process; data preprocesing, model building, and experimentation/forwarding. In doing so, we make it
        quicker to develop model and data pipelines, and easier to follow the processes within.


    The example below represents a simple, explicit implementation of a differentiable model using the HBV backbone.
    In this case we have the following steps
    1.  **Load a configuration file**: Using Hydra and OmegaConf packages, we can convert our `..config.yaml` into a dictionary
        object `config`. For example, if your config file contains `mode: train`, the dictionary yields
        `config['mode'] == 'train'`. However, config can also contain sub-dictionaries. Take
        ```yaml
        training: 
            start_time: 1999/10/01
        ```
        for instance. In these cases, access the subdict like so `config['training']['start_time'] == '1999/10/01'`.
    
    2.  **Load in data**: At this step, we load in and process our data as a dictionary of attributes and feature datasets
        that are used by the HBV model. In general, this dataset dict is supplied by the user, and should meet
        minimum requirements established in `docs/getting_started_guide.md`.

        In this example, we take a small, arbitrarily selected sample of the data to illustrate the modeling process.

    3.  **Initialize sub-models**: Next, we initialize the physics model and neural network our differentiable model
        will use, HBV and LSTM in this case.

    4.  **Create a differentiable model**: Now, the sub-models are linked together by the DeltaModel wrapper. This
        wrapper interfaces models using the desired modality, i.e., forwarding the LSTM to generate parameters that
        it then feeds with forcing variables to HBV to generate predictions. 

    5.  **Forward/Experiment**: With the differentiable model created, it can be forwarded (as demonstrated
        below), or trained/tested/applied in any user-defined experiments.

In [19]:
import sys
sys.path.append('../../deltaModel')  # Add the root directory of deltaModel

from example import load_config 
from hydroDL2.models.hbv.hbv import HBVMulTDET as hbv
from deltaModel.models.neural_networks import init_nn_model
from deltaModel.core.data.dataset_loading import get_dataset_dict # Eventually a hydroData import
from deltaModel.models.differentiable_model import DeltaModel as dHBV
from deltaModel.core.data import take_sample



CONFIG_PATH = '../conf/dhbv_config.yaml'



# 1. Load configuration dictionary of model parameters and options
config = load_config(CONFIG_PATH)

# 2. Setup a dataset dictionary of NN and physics model inputs.
# Take a sample to reduce size on GPU.
dataset = get_dataset_dict(config, train=True)
dataset_sample = take_sample(config, dataset, days=730, basins=100)

# 3. Initialize physical model and neural network
phy_model = hbv(config['dpl_model'])
nn = init_nn_model(phy_model, config['dpl_model'])

# 4. Create the differentiable model dHBV: 
# a torch.nn.Module that describes how nn is linked to the physical model.
dpl_model = dHBV(phy_model, nn)

## From here, dpl_model can be run or trained as any torch.nn.Module model in a
## standard training loop.

# 5. For example, to forward:
output = dpl_model.forward(dataset_sample)



print(f"Streamflow predictions for {output['flow_sim'].shape[0]} days and {output['flow_sim'].shape[1]} basins: Showing the first 5 days for 5 basins \n {output['flow_sim'][:5,:5]}")  # TODO: Add a visualization here.

Streamflow predictions for 365 days and 100 basins: Showing the first 5 days for 5 basins 
 tensor([[[1.3719e-05],
         [1.6588e-05],
         [1.3494e-05],
         [1.8438e-05],
         [1.6097e-05]],

        [[3.3860e-05],
         [3.8847e-05],
         [3.3249e-05],
         [4.1722e-05],
         [3.7960e-05]],

        [[5.3675e-05],
         [5.9653e-05],
         [5.2746e-05],
         [6.2785e-05],
         [5.8545e-05]],

        [[7.0703e-05],
         [7.6776e-05],
         [6.9597e-05],
         [7.9663e-05],
         [7.5593e-05]],

        [[8.4114e-05],
         [8.9671e-05],
         [8.2974e-05],
         [9.2033e-05],
         [8.8523e-05]]], device='cuda:0', grad_fn=<SliceBackward0>)



---

## 2. Training a Model

Now that we can build a differentiable model, we will train $\delta$ HBV (1.0 or 1.1p) and expose critical steps in the
process, particularly within the model trainer.

### 2.1 Load Config and Dataset

Once again, we begin by loading in the config dict and CAMELS dataset.

Be sure to change `CONFIG_PATH` to the HBV version you want to use.

<!-- This code instantiates a model Trainer which will train or test a model per the user's specification in the config. Note
that `__main__.py` is trimmed-down here to illustrate it's primary objective. The trainer performs the following
functions:
- CAMELS data will be loaded and preprocessed,
- A differenial model object with the HBV1.1p backbone will be created, and 
- An optimizer and loss function will be initialized. -->

In [21]:
import sys
sys.path.append('../../deltaModel')  # Add the root directory of deltaModel

from example import load_config 
from deltaModel.core.data.dataset_loading import get_dataset_dict # Eventually a hydroData import



CONFIG_PATH = '../conf/dhbv_config.yaml'



# Load configuration dictionary of model parameters and options
config = load_config(CONFIG_PATH)

# Setup a dataset dictionary of NN and physics model inputs.
# Take a sample to reduce size on GPU.
dataset = get_dataset_dict(config, train=True)

### 2.2 Initialize a $\delta$ HBV Differentiable Model, Optimizer, and Loss Function

These are the auxillary tasks completed by the Trainer before beginning the training loop.


<!-- We use the Adadelta optimizer from PyTorch, feeding it both learnable model
parameters and a learning rate from the config file.


Dynamically load the loss function identified in the config (RMSE for
dHBV 1.0 and NSE for dHBV 1.1p). -->

In [31]:
import torch

from hydroDL2.models.hbv.hbv import HBVMulTDET as hbv
from deltaModel.models.loss_functions import get_loss_fn
from deltaModel.models.neural_networks import init_nn_model
from deltaModel.models.differentiable_model import DeltaModel as dHBV



# Initialize physical model and neural network
phy_model = hbv(config['dpl_model'])
nn = init_nn_model(phy_model, config['dpl_model'])

# Create the differentiable model dHBV: 
# a torch.nn.Module that describes how nn is linked to the physical model.
dpl_model = dHBV(phy_model, nn)
print(f"Here is our dHBV framework: \n ----- \n {dpl_model}")

# Init an Adadelta optimizer
optimizer = torch.optim.Adadelta(
    dpl_model.parameters(),
    lr=config['dpl_model']['nn_model']['learning_rate']
    )

# init a loss function
loss_fn = get_loss_fn(config, dataset['obs'])


Here is our dHBV framework: 
 ----- 
 DeltaModel(
  (phy_model): HBVMulTDET()
  (nn_model): CudnnLstmModel(
    (linearIn): Linear(in_features=38, out_features=256, bias=True)
    (lstm): CudnnLstm()
    (linearOut): Linear(in_features=256, out_features=210, bias=True)
  )
)


### 2.4 Train the Model

Below we use a basic training loop to train the LSTM in $\delta$ HBV to optimize HBV's parameters and improve its
streamflow predictions.


#### Key Steps in the Training Loop
1. **Calculate Training Parameters**  
   The `calc_training_params` function calculates the training settings:
   - `n_sites`: The number of unique locations/sites in the dataset.
   - `n_minibatch`: The number of samples to process per epoch.
   - `n_timesteps`: The number of timesteps per sample.

2. **Epoch Loop**  
   Each epoch represents one full cycle through the training data. For each epoch:
   - `total_loss` is reset to track the total error across all batches within the epoch.

3. **Batch Loop**  
   Within each epoch, the code processes data in smaller chunks (minibatches) to improve training efficiency and avoid
   oversaturation of GPU VRAM. 
   
   For each batch:
   - **Sample Data**: `take_sample_train` randomly selects a sample of training data for the batch.
   - **Forward Pass**: The model processes the input data to produce predictions.
   - **Calculate Loss**: `loss_fn` compares predictions to observed values to calculate the error for the batch.
   - **Backward Pass and Optimization**: 
     - `loss.backward()` computes gradients to adjust the model’s parameters.
     - `optimizer.step()` updates the LSTM parameters.
     - `optimizer.zero_grad()` resets gradients for the next batch.


In [32]:
import tqdm
from core.data import calc_training_params, take_sample_train
from core.utils import save_model




# Batch size, number of training samples per epoch, and number of timesteps
n_sites, n_minibatch, n_timesteps = calc_training_params(
    dataset['inputs_nn_scaled'],
    config['train_t_range'],
    config
    )

# Start of training.
for epoch in range(1, config['train']['epochs'] + 1):
    total_loss = 0.0 # Initialize epoch loss to zero.

    # Work through training data in batches.
    prog_str = f"Epoch {epoch}/{config['train']['epochs']}"
    
    for i in tqdm.tqdm(range(1, n_minibatch + 1), desc=prog_str,
                       leave=False, dynamic_ncols=True):
        
        # Take a sample of the training data for the batch.
        dataset_sample = take_sample_train(config, dataset, n_sites, n_timesteps)

        # Forward pass through dPL model.
        predictions = dpl_model.forward(config, dataset_sample)        
        
        # Calculate loss.
        loss = loss_fn(config,
                       predictions,
                       dataset_sample['obs'],
                       igrid=dataset_sample['iGrid']
                       )
                                   
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
    
    avg_loss = total_loss / n_minibatch + 1
    print(f"Avg model loss after epoch {epoch}: {avg_loss}")

    # Save the model every save_epoch (set in the config).
    model_name = config['dpl_model']['phy_model']['model_name']
    if epoch % config['train']['save_epoch'] == 0:
        save_model(config, dpl_model, model_name, epoch)

KeyError: 'phy_model'


---

## 3 Test a Model

If you have run testing on a trained model and want to view the results, you can find a `mstd.csv` file in your
`results/` directory, which will present performance statistics for the respective model. 

*A testing tutorial and Graphical visualizations of model output will be supported in a future updated.*
